In [ ]:
import pandas as pd
from datetime import datetime
import os

In [ ]:
today = datetime.today().strftime("%d%m%Y")
path = rf'C:\Users\muhammadadlan\Downloads\customer_ultimate_beta\raw_data'

In [ ]:
transformed_path = rf'C:\Users\muhammadadlan\Downloads\customer_ultimate_beta\transformed_data'

directory_path = rf'{transformed_path}\{today}'

os.makedirs(directory_path, exist_ok=True)

print(f"Directory '{directory_path}' created.")

ETL:

Customer

In [ ]:
customer_df = pd.read_csv(rf'{transformed_path}\{today}\customer_test_v2.csv', dtype={'id':str}, index_col=0)

In [ ]:
import pandas as pd

# Assuming your DataFrame is called 'customer_data'
columns_to_drop = [col for col in customer_df.columns if 'devices_' in col and int(col.split('_')[1]) > 0]

customer_df = customer_df.drop(columns=columns_to_drop)

# Sort the dataframe
customer_df = customer_df.sort_values(['id', 'version', 'lastUpdate'], ascending=[True, True, True])

# Drop duplicates based on the 'id' column, keeping the last occurrence
customer_df = customer_df.drop_duplicates(subset=['id'], keep='last')

customer_df = customer_df.loc[:, ~customer_df.columns.str.contains('^Unnamed')]

customer_df.to_csv(rf'transformed_data\{today}\customer_v2.csv')


In [ ]:
# Count the number of unique ids
num_unique_ids = customer_df['id'].nunique()

print(f'There are {num_unique_ids} unique ids in the file.')

Savings Acc:

In [ ]:
sa_df= pd.read_csv(rf'{path}\sa_full.csv')

In [ ]:
size = sa_df.shape
print('The size of the dataset is', size)

In [ ]:
sa_df.isnull().sum()

In [ ]:
sa_df = sa_df.loc[:, ~sa_df.columns.str.contains('^Unnamed')]

sa_df.to_csv(rf'transformed_data\{today}\sa_v2.csv')

Savings Pot:

In [ ]:
sp_df= pd.read_csv(rf'{path}\sp_full.csv')

In [ ]:
size = sp_df.shape
print('The size of the dataset is', size)

In [ ]:
sp_df.isnull().sum()

In [ ]:
sp_df = sp_df.loc[:, ~sp_df.columns.str.contains('^Unnamed')]

sp_df.to_csv(rf'transformed_data\{today}\sp_v2.csv')

Payments:

In [ ]:
payments_df= pd.read_csv(rf'{path}\payments_full.csv')

In [ ]:
size = payments_df.shape
print('The size of the dataset is', size)

In [ ]:
payments_df.isnull().sum()

In [ ]:
payments_df = payments_df.loc[:, ~payments_df.columns.str.contains('^Unnamed')]

payments_df.to_csv(rf'transformed_data\{today}\payments_v2.csv')

FTV:

In [ ]:
ftv_df = pd.read_csv(rf'{path}\ftv_full.csv', dtype={'customerId':str})

In [ ]:
size = ftv_df.shape
print('The size of the dataset is', size)

In [ ]:
ftv_df.isnull().sum()

In [ ]:
ftv_df = ftv_df.loc[:, ~ftv_df.columns.str.contains('^Unnamed')]

In [ ]:
ftv_df = ftv_df.sort_values(['tmCreatedAt'], ascending=[True])
ftv_df = ftv_df.drop_duplicates(subset='customerId', keep='last')

ftv_df.to_csv(rf'transformed_data\{today}\ftv_v2.csv')

Reversal:

In [ ]:
reversal_df = pd.read_csv(rf'{path}\reversal_full.csv', dtype={'customerId':str})

In [ ]:
size = reversal_df.shape
print('The size of the dataset is', size)

In [ ]:
reversal_df = reversal_df.loc[:, ~reversal_df.columns.str.contains('^Unnamed')]

In [ ]:
reversal_df.to_csv(rf'transformed_data\{today}\reversal_v2.csv')

Address:

In [ ]:
address_df = pd.read_csv(rf'{path}\address_full.csv')

In [ ]:
size = address_df.shape
print('The size of the dataset is', size)

In [ ]:
address_df.isnull().sum()

In [ ]:
# Move the CustomerID column to be the first column
customer_id = address_df.pop('customerId')
address_df.insert(0, 'customerId', customer_id)

# Print the modified DataFrame
print(address_df.head())

In [ ]:
# Convert the values in the "subdivision" column to uppercase
address_df['subdivision'] = address_df['subdivision'].str.upper()

# Replace long state names with abbreviations
address_df['subdivision'] = address_df['subdivision'].str.replace("WILAYAH PERSEKUTUAN KUALA LUMPUR", "WPKL")
address_df['subdivision'] = address_df['subdivision'].str.replace("Kuala Lumpur", "WPKL")

address_df['subdivision'] = address_df['subdivision'].str.replace("WILAYAH PERSEKUTUAN PUTRAJAYA", "WP PUTRAJAYA")
address_df['subdivision'] = address_df['subdivision'].str.replace("WILAYAH PERSEKUTUAN LABUAN", "WP LABUAN")

address_df['subdivision'] = address_df['subdivision'].str.replace("Selangor", "SELANGOR")
address_df['subdivision'] = address_df['subdivision'].str.replace("SELANGOR DARUL EHSAN", "SELANGOR")

address_df['subdivision'] = address_df['subdivision'].str.replace("Melaka", "MELAKA")

address_df['subdivision'] = address_df['subdivision'].str.replace("SARAWAK", "SARAWAK")

address_df['subdivision'] = address_df['subdivision'].str.replace("Johor", "JOHOR")
address_df['subdivision'] = address_df['subdivision'].str.replace("JCHOR", "JOHOR")

address_df['subdivision'].astype('category').value_counts()

In [ ]:

# Perform the value counts operation and assign the result to a new DataFrame
counts_address_df = address_df['subdivision'].astype('category').value_counts().reset_index()

# Rename the columns to be more descriptive
counts_address_df.columns = ['subdivision', 'count']

# Add a percentage column
counts_address_df['percentage'] = counts_address_df['count'].apply(lambda x: round(x/counts_address_df['count'].sum() * 100, 2))

# Print the modified DataFrame
print(counts_address_df)

In [ ]:
# Sort the DataFrame by the 'type' column in descending order
address_df = address_df.sort_values(by='type', ascending=False)

address_df.head(2)

In [ ]:
address_df = address_df.sort_values(by=['customerId', 'type'])

df_residential = address_df[address_df['type'] == "RESIDENTIAL"]

df_residential.to_csv(rf'transformed_data\{today}\residential.csv', index=False)

In [ ]:
address_df = address_df.sort_values(by=['customerId', 'type'])

df_mailing = address_df[address_df['type'] == "MAILING"]

df_mailing.to_csv(rf'transformed_data\{today}\mailing.csv', index=False)

In [ ]:
import pandas as pd

# read in the dataframes
df_residential = pd.read_csv(rf'transformed_data\{today}\residential.csv')
df_mailing = pd.read_csv(rf'transformed_data\{today}\mailing.csv')

address_cleaned_df = pd.merge(df_residential, df_mailing, left_on='customerId', right_on='customerId', how='outer')

In [ ]:
import re

address_cleaned_df = address_cleaned_df.applymap(lambda x: re.sub(r'(?<!#)\s+', ' ', str(x)))

In [ ]:
address_cleaned_df = address_cleaned_df.loc[:, ~address_cleaned_df.columns.str.contains('^Unnamed')]

In [ ]:
address_cleaned_df = address_cleaned_df.sort_values(by='lastUpdated_x', ascending=True)

# Drop duplicates based on the 'id' column, keeping the last occurrence
address_cleaned_df = address_cleaned_df.drop_duplicates(subset=['customerId'], keep='last')

address_cleaned_df.to_csv(rf'transformed_data\{today}\address_cleaned.csv', index=False)

Employment:

In [ ]:
employment_df = pd.read_csv(rf'{path}\employment_full.csv')

In [ ]:
size_emp = employment_df.shape
print('The size of the dataset is', size_emp)

In [ ]:
employment_df.isnull().sum()

In [ ]:
print(employment_df.head())

In [ ]:
# Convert the values in the "employer" column to uppercase
employment_df['employer'] = employment_df['employer'].str.upper()

# Check if the 'employer' column contains 'Rajhi' or 'Rahji'
mask1 = employment_df['employer'].str.contains('RAJHI', na=False)
mask2 = employment_df['employer'].str.contains('RAHJI', na=False)

# Replace all occurrences of 'RAJHI' and 'RAHJI' with 'ARBM'
employment_df.loc[mask1, 'employer'] = 'ARBM'
employment_df.loc[mask2, 'employer'] = 'ARBM'

# Check if the 'employer' column contains 'TCT'
mask3 = employment_df['employer'].str.contains('TCT', na=False)

# Replace all occurrences of 'TCT' with 'TCT TRADING SDN BHD'
employment_df.loc[mask3, 'employer'] = 'TCT TRADING SDN BHD'

In [ ]:
employment_df['employer'].astype('category').value_counts()

In [ ]:
# Move the CustomerID column to be the first column
customer_id = employment_df.pop('customerId')
employment_df.insert(0, 'customerId', customer_id)

# Print the modified DataFrame
print(employment_df.head())

In [ ]:
import re

employment_df = employment_df.applymap(lambda x: re.sub(r'(?<!#)\s+', ' ', str(x)))

In [ ]:
employment_df = employment_df.loc[:, ~employment_df.columns.str.contains('^Unnamed')]

employment_df.to_csv(rf'transformed_data\{today}\employment_cleaned.csv', index=False)

Profile:

In [ ]:
# Load CSV
profile_df = pd.read_csv(rf'{path}\profile_full.csv')

In [ ]:
size_profile = profile_df.shape
print('The size of the dataset is', size_profile)

In [ ]:
profile_df.isnull().sum()

In [ ]:
#dropping columns
profile_df = profile_df.drop(['id','religion','createdOn', 'lastUpdated'], axis=1)
profile_df.shape

In [ ]:
# Rename the column from 'bnmCcc' to 'Bumi'
profile_df = profile_df.rename(columns={'bnmCcc': 'Bumi'})
# Replace values in the 'Bumi' column
profile_df['Bumi'] = profile_df['Bumi'].replace({'Individual Bumi': True, 'Individual Non-Bumi': False})

print(profile_df.head())

In [ ]:
# Move the CustomerID column to be the first column
customer_id = profile_df.pop('customerId')
profile_df.insert(0, 'customerId', customer_id)

# # convert the customer ID column to a string
# df['customerId'] = df['customerId'].astype(object)

# Print the modified DataFrame
print(profile_df.head())

In [ ]:
profile_df = profile_df.loc[:, ~profile_df.columns.str.contains('^Unnamed')]

profile_df.to_csv(rf'transformed_data\{today}\profile_cleaned.csv', index=False)

In [ ]:
profile_df.shape

In [ ]:

# Filter the DataFrame to include only rows that meet the criteria
single_malay_bumi = profile_df[(profile_df['maritalStatus'] == 'Single') & (profile_df['ethnicity'] == 'Malay') & (profile_df['Bumi'] == True)]

# Count the number of rows in the filtered DataFrame
count = len(single_malay_bumi)

# Print the count
print('There are ' + str(count) + ' Individuals')

In [ ]:
# Filter the DataFrame to include only rows that meet the criteria
married_malay_bumi = profile_df[(profile_df['maritalStatus'] == 'Married') & (profile_df['ethnicity'] == 'Malay') & (profile_df['Bumi'] == True)]

# Count the number of rows in the filtered DataFrame
count = len(married_malay_bumi)

# Print the count
print('There are ' + str(count) + ' Couples')

In [ ]:
# Group the DataFrame by 'maritalStatus', 'ethnicity', and 'Bumi', and count the number of rows in each group
count_profile_df = profile_df.groupby(['maritalStatus', 'ethnicity', 'Bumi']).size().reset_index(name='count')

# Print the new table
print(count_profile_df)

In [ ]:
import matplotlib.pyplot as plt

# Filter the count dataframe to only include rows where maritalStatus is "single"
single_df = count_profile_df[count_profile_df['maritalStatus'] == 'Single']

# Plot the pie chart
plt.pie(single_df['count'], labels=single_df['ethnicity'], autopct='%1.1f%%')
plt.title('Distribution of Ethnicities Among Single Individuals')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Group the DataFrame by 'maritalStatus', 'ethnicity', and 'Bumi', and count the number of rows in each group
count_df = profile_df.groupby(['maritalStatus', 'ethnicity', 'Bumi']).size().reset_index(name='count')

# Create a pivot table to make it easier to plot
pivot_table = count_df.pivot_table(index=['maritalStatus', 'ethnicity'], columns='Bumi', values='count', fill_value=0)

# Create the stacked bar chart
ax = pivot_table.plot(kind='bar', stacked=True, figsize=(10, 6))

# Set the title and axis labels
ax.set_title('Counts by Marital Status, Ethnicity, and Bumi')
ax.set_xlabel('Marital Status, Ethnicity')
ax.set_ylabel('Count')

# Add a legend
ax.legend(title='Bumi', bbox_to_anchor=(1, 1))

# Show the plot
plt.show()

In [ ]:
# Group the dataframe by gender and count the number of occurrences
status_counts = profile_df['ethnicity'].value_counts()

# Create a pie chart with the gender counts
plt.pie(status_counts, labels=status_counts.index, autopct='%1.1f%%')
plt.title('Status Distribution')
plt.legend(title='Status Count', loc='upper right', bbox_to_anchor=(1, 0, 0.5, 1))
plt.show()

CRR:

In [ ]:
crr = pd.read_csv(rf'{path}\crr_full.csv')

In [ ]:
# Rename columns
crr = crr.rename(columns={'customerId':'CIC','score': 'crr_score', 'rating':'crr_rating', 'createdAt':'crr_timestamp'})

# Define the columns you want to select
column_crr = ['CIC', 'crr_score', 'crr_rating', 'crr_timestamp']

# Select these columns from the DataFrame
crr = crr[column_crr]

# Convert 'timestamp' to datetime if it's not already
crr['crr_timestamp'] = pd.to_datetime(crr['crr_timestamp'])

# Sort by 'timestamp' in descending order so the latest timestamps are first
crr = crr.sort_values('crr_timestamp', ascending=False)

# Drop duplicates, keeping the first (latest) entry for each 'CIC'
crr = crr.drop_duplicates(subset='CIC', keep='first')

In [ ]:
crr = crr.loc[:, ~crr.columns.str.contains('^Unnamed')]

crr.to_csv(rf'transformed_data\{today}\crr_cleaned.csv', index=False)

ComplyAdv:

In [ ]:
complyadv = pd.read_csv(rf'{path}\complyadv_full.csv')

In [ ]:
complyadv.isnull().sum()

In [ ]:
# Rename columns
complyadv = complyadv.rename(columns={'customerId':'CIC','createdAt':'complyAdv_timestamp'})

# Define the columns you want to select
column_complyadv = ['CIC', 'nameScreeningMatch', 'matchStatus', 'totalHits', 'totalBlacklistHits','complyAdv_timestamp']

# Select these columns from the DataFrame
complyadv = complyadv[column_complyadv]

# Convert 'timestamp' to datetime if it's not already
complyadv['complyAdv_timestamp'] = pd.to_datetime(complyadv['complyAdv_timestamp'])

# Sort by 'timestamp' in descending order so the latest timestamps are first
complyadv = complyadv.sort_values('complyAdv_timestamp', ascending=False)

# Drop duplicates, keeping the first (latest) entry for each 'CIC'
complyadv = complyadv.drop_duplicates(subset='CIC', keep='first')

In [ ]:
complyadv = complyadv.loc[:, ~complyadv.columns.str.contains('^Unnamed')]

complyadv.to_csv(rf'transformed_data\{today}\complyadv_cleaned.csv', index=False)

Balances:

In [ ]:
balance_df = pd.read_csv(rf'{transformed_path}\{today}\balance_cleaned.csv')

In [ ]:
						
# # Extract and rename relevant columns in balances_df
# balance_df = balance_df[[
#     'balance_created_balance_account_id',
#     'balance_created_balance_amount',
#     'balance_created_balance_denomination',
#     'balance_created_balance_total_credit',
#     'balance_created_balance_total_debit',
#     'balance_created_balance_value_time',
#     'balance_created_balance_account_address',
#     'timestamp'
#     ]].rename(columns={
#         'balance_created_balance_account_id': 'tm_account_id', 
#         'balance_created_balance_amount': 'ledger_balance',
#         'balance_created_balance_total_credit': 'total_credit',
#         'balance_created_balance_total_debit': 'total_debit',
#         'balance_created_balance_value_time': 'last_transaction_date',
#         'balance_created_balance_account_address': 'account_address',
#         'timestamp':'balance_timestamp'
#         })


In [ ]:
balance_df = balance_df.sort_values(by=['tm_account_id', 'account_address', 'last_transaction_date'])

balance_df_default = balance_df[balance_df['account_address'] == "DEFAULT"]

balance_df_default = balance_df_default.drop_duplicates(subset='tm_account_id', keep='last')

In [ ]:
balance_df_default.isnull().sum()

In [ ]:
balance_df_default = balance_df_default.loc[:, ~balance_df_default.columns.str.contains('^Unnamed')]

In [ ]:
balance_df_default.to_csv(rf'transformed_data\{today}\balance_cleaned.csv', index=False)